In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
import random

In [129]:
# initialization
subjects = 200
types = 3
usePCA = True
useMDA = False
# dataset_file = 'Data/pose.mat'
dataset_file = 'Data/data.mat'
# dataset_file = 'Data/illumination.mat'
dataset = ''

In [130]:
# load the dataset
data_ = sio.loadmat(dataset_file)

In [131]:
if 'pose' in dataset_file:
    data = data_.get('pose')
    dataset = 'pose'
elif 'illumination' in dataset_file:
    data = data_.get('illum')
    dataset = 'illum'
else:
    data = data_.get('face')
    dataset = 'face'

In [132]:
# flatten the dataset
if dataset == 'pose':
    flattened = np.zeros(shape=(subjects*types, data.shape[0]*data.shape[1]))
elif dataset == 'illum':
    flattened = np.zeros(shape=(subjects*types, data.shape[0]))
else:
    flattened = np.zeros(shape=(subjects*types, data.shape[0]*data.shape[1]))

c = 0
d = 0
for i in range(flattened.shape[0]):
    if c == types:
        c = 0
        d += 1
    if dataset == 'pose':
        temp = data[:,:,c,d]
        flattened[i] = temp.flatten()
    elif dataset == 'face':
        temp = data[:,:,i]
        flattened[i] = temp.flatten()
    elif dataset == 'illum':
        flattened[i] = data[:,c,d]
    # y[i] = d
    c += 1


In [133]:
# def doPCA(flattened, dim):
#     pca = PCA(dim)
#     projected = pca.fit_transform(flattened)
#     return projected

In [134]:
# # Perform PCA if true
# if usePCA:
#     pca = PCA().fit(flattened)
#     plt.figure()
#     plt.xlabel('Dimensions')
#     plt.ylabel('Variance Retention')
#     plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3)
#     min_dim = (np.where(pca.explained_variance_ratio_.cumsum() > 0.95))[0][0]
#     print('Minimum dimensions required for 95% retention ', min_dim)
#     projected = doPCA(flattened, min_dim)

In [135]:
# print('Before dimension reduction shape = ', flattened.shape)
# print('After dimension reduction shape = ', projected.shape)

In [136]:
dataset

'face'

In [138]:
# noise = {'pose':0.987}
noise = {'pose':0.93, 'face':0.999}

In [139]:
from tqdm import tqdm
mu_class = [] # emperical mean of each class
cov_class = [] # emperical covariace of each class

mu_not = np.zeros(shape=(flattened.shape[1]))
sigma_b = np.zeros(shape=(flattened.shape[1], flattened.shape[1]))
sigma_w = np.zeros(shape=(flattened.shape[1], flattened.shape[1]))
prob = 1/subjects

for i in range(subjects):
    # determine class as per the data
    start = i*types
    end = (i+1)*types
    # print(start, end)
    
    temp = flattened[start:end]
    mean = np.mean(temp, axis=0)
    mu_class.append(mean)

    cov = np.zeros(shape=(temp.shape[1], temp.shape[1]))
    for k in range(types):
        mat = (temp[k] - mean).reshape(temp.shape[1], 1)
        cov = cov + np.dot(mat, mat.T)
    # mean of covariance
    cov = cov / types
    # add noise

    cov += noise.get(dataset, 0.8)*np.identity(cov.shape[0])
    cov_class.append(cov)
    print(np.linalg.det(cov))
    # break

    # emperical mean mu_not
    mu_not += prob*mean

    #sigma_w
    sigma_w += prob*cov

# calculate the sigma_b
for i in range(subjects):
    mat = (mu_class[i] - mu_not).reshape(flattened.shape[1], 1)
    sigma_b += prob*np.dot(mat, mat.T)

98.99456914241867
94.65618955191484
120.92340485415028
244.1201125144308
140.92459301001247
103.34760701571844
137.9865160172802
459.1202368593323
76.71745911627767
75.31896369177547
129.33129226194282
101.40048176902036
176.59874516876792
377.03180432903804
168.92179270021578
108.74583291016359
73.95545773577902
87.29079347327583
85.85010989091364
598.7078176112583
229.6602309570126
129.79773968217677
126.13831421147175
582.2686249433084
214.43416964871693
538.7558262921663
242.02880016073567
77.60257631447878
263.4496567746993
228.94679039163933
203.6300578856702
82.68281831112026
64.77434774109511
181.06247450099366
80.00988478964203
115.90643547880505
167.81755846825496
1433.5864425142622
470.4138114636537
99.20791861671597
119.40372800514209
240.71895224800974
75.05360905122083
161.271303887164
313.5740187729072
208.03207519980285
181.46946423589694
136.31843920394996
51.80265994700934
81.56696267397005
195.65533588784922
237.1402909155222
222.07223337956682
443.7150359679334
371.

In [141]:
# # pose   
# b = sigma_b + 0.724*np.identity(cov.shape[0])   #0.745999
# np.linalg.det(b)

# face
b = sigma_b + 0.945*np.identity(cov.shape[0])
np.linalg.det(b)



25.440157719622196

In [142]:
# a = sigma_w #+ np.identity(cov_class[0].shape[0])
a = np.dot(np.linalg.inv(sigma_w), b)

In [143]:

# for i in range(subjects):
#     mat = (mu_class[i] - mu_not).reshape(flattened.shape[1], 1)
#     sigma_b += prob*np.dot(mat, mat.T)

In [144]:
val, vec = np.linalg.eig(a)

In [145]:
val

array([7.70957982, 4.56649305, 3.57955419, 2.39928485, 2.31974468,
       2.12757939, 0.10758501, 1.82186662, 1.76534184, 1.70778498,
       1.6014136 , 1.57568863, 0.42685936, 1.50460557, 1.45745142,
       1.37983694, 0.61052613, 1.36275052, 1.33597531, 1.3234253 ,
       0.64872466, 1.29564514, 1.28050148, 1.25130504, 0.69238929,
       1.22486674, 1.22316617, 1.20430274, 1.18299162, 1.17724338,
       1.15377782, 1.14645848, 0.74518611, 1.13537888, 0.75269341,
       1.11800471, 1.11422122, 1.1081227 , 1.11087304, 1.09885005,
       1.09468217, 1.08553592, 1.07950445, 1.07748347, 0.77004629,
       0.77519389, 0.77909364, 0.78511493, 0.79190924, 0.79305827,
       0.8075819 , 0.81540528, 0.82042677, 0.81695352, 0.81841645,
       1.06846564, 1.07152739, 1.05831823, 1.05333209, 0.82390709,
       1.05179969, 0.82791803, 0.8313994 , 1.04436604, 1.04361718,
       1.04275484, 1.03970693, 0.83921917, 1.03671184, 1.03494042,
       0.84060836, 0.84184799, 0.84467707, 1.02967679, 0.85087

In [72]:
val[0]

(423145.6879954041+0j)

In [146]:
vec[0]

array([ 3.57757839e-02, -3.24453189e-02,  2.29604262e-02, -3.96572935e-02,
        3.97997267e-03, -1.45317922e-02, -2.43322378e-02, -1.94980155e-02,
       -3.59646127e-02,  2.19187500e-02,  3.49526548e-03,  1.37735149e-02,
        8.19477500e-03, -1.49545961e-02,  5.53344639e-04, -2.36803094e-02,
       -4.21063535e-03,  3.30439023e-04, -1.01662682e-02,  1.32883087e-02,
        8.10518176e-03,  1.91207559e-02, -3.36824364e-02, -6.87866920e-03,
        9.94912154e-03,  2.38346738e-02, -9.94525342e-03, -1.19204369e-02,
        7.81709490e-03,  1.06855609e-02,  6.28964099e-03,  2.00766609e-03,
        4.70701850e-04, -1.70269486e-02, -5.86230578e-03,  7.91888392e-04,
        4.65040683e-03,  8.66268415e-03,  2.84591003e-03,  1.53082669e-02,
       -2.41889228e-03, -5.10779819e-03, -1.72193251e-02, -9.53942225e-03,
       -9.46535446e-03,  1.64128859e-03, -1.28145428e-02,  2.58085390e-02,
        3.55591111e-03, -1.11265042e-02,  1.89597025e-02,  1.69810422e-02,
       -4.23988358e-03, -

In [ ]:
from sklearn.discriminant_analysis import  LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(solver='eigen')

In [ ]:
lda = lda.fit(flattened, y)

In [ ]:
plt.plot(lda.explained_variance_ratio_.cumsum(), lw=3)

In [ ]:
mat = lda.fit_transform(flattened, y)

In [147]:
idx = val.argsort()[::-1]

In [148]:
val_ = val[idx]
val_

array([7.70957982, 4.56649305, 3.57955419, 2.39928485, 2.31974468,
       2.12757939, 1.82186662, 1.76534184, 1.70778498, 1.6014136 ,
       1.57568863, 1.50460557, 1.45745142, 1.37983694, 1.36275052,
       1.33597531, 1.3234253 , 1.29564514, 1.28050148, 1.25130504,
       1.22486674, 1.22316617, 1.20430274, 1.18299162, 1.17724338,
       1.15377782, 1.14645848, 1.13537888, 1.11800471, 1.11422122,
       1.11087304, 1.1081227 , 1.09885005, 1.09468217, 1.08553592,
       1.07950445, 1.07748347, 1.07152739, 1.06846564, 1.05831823,
       1.05333209, 1.05179969, 1.04436604, 1.04361718, 1.04275484,
       1.03970693, 1.03671184, 1.03494042, 1.02967679, 1.02615941,
       1.02479975, 1.02223587, 1.01905565, 1.01562359, 1.01336886,
       1.01172152, 1.00879813, 1.00667834, 1.00596059, 1.00266829,
       1.00115676, 1.0002597 , 0.99978362, 0.99818041, 0.99468552,
       0.99440461, 0.99292763, 0.99164733, 0.98879069, 0.98873238,
       0.98696587, 0.98629933, 0.9853176 , 0.98448125, 0.98299

In [149]:
vec_  = vec[:, idx]

In [150]:
vec_.shape

(504, 504)

In [151]:
dim = 199
vec__ = vec_[:,:dim]

In [152]:
vec__.shape

(504, 199)

In [153]:
f = np.dot(flattened, vec__)

# TEST ON FACE DATA

In [154]:
projected = f

In [155]:
projected.shape

(600, 199)

In [156]:
data.shape

(24, 21, 600)

In [157]:
useMDA = True
if (useMDA):
    training_data = []
    testing_data = []
    c = 1 # counter to pick random sample from either illumination or 
    for i in range(0, data.shape[2], 3):
        training_data.append(projected[i])
        if c % 2 == 0:
            training_data.append(projected[i+1])
            testing_data.append(projected[i+2])
        else:
            training_data.append(projected[i+2])
            testing_data.append(projected[i+1])
        c += 1

In [158]:
if(useMDA):
    mu = []
    cov = []
    for i in range(0, len(training_data), 2):
        mean = ((training_data[i] + training_data[i+1]) / 2).reshape(1, projected.shape[1])
        cov1 = np.matmul((training_data[i]-mean).T, training_data[i]-mean)
        cov2 = np.matmul((training_data[i+1]-mean).T, training_data[i+1]-mean)
        noise = 1.0*np.identity(cov1.shape[0]) #0.24
        cov_ = (cov1 + cov2)/2 + noise
        print(np.linalg.det(cov_))
        # break
        cov.append(cov_)
        mu.append(mean)
        if np.linalg.det(cov_) == 0 or np.linalg.det(cov_) == 0.0:
            print('alert - zero determinant')
    

4.649379822361784
3.6145086456934328
9.149803066954837
6.793423542608418
5.127702178287167
4.038220327909681
6.062699150384918
4.324983912189782
5.326610993142969
2.9110140356356697
7.409180287333318
5.367608435455189
7.816695848654923
8.786732815635073
7.748799414515975
4.602547160738278
5.994280203758261
2.8125514045702427
4.296643032596023
6.55203882057608
9.501115273789779
3.398748972203964
4.538254803134906
5.195700020086381
4.277718547329315
7.133609180203286
4.966321420733529
3.686942616020682
10.035068412627131
4.274837197684763
3.877599810080514
4.144939749421682
5.656129614150467
3.6692938469324323
6.239349160538938
3.9497503601998547
3.8796624886575275
8.215539760538553
9.415507707324318
4.412448781843359
4.2468908597372526
4.799430189932512
4.4901435834617285
4.488853831198663
10.723549694160436
10.987655416191009
7.0729574341624355
6.197447086825862
4.124455902203039
3.6239580263236952
6.776948681129207
7.981544274300526
5.394416455010537
8.520604873823439
8.54505714678101

In [159]:
score = 0
training_size = int((2/3)*data.shape[2])
testing_size = int((1/3)*data.shape[2])
for i in range(testing_size):
    likelihood_list = []
    for j in range(testing_size):
        likelihood = (-0.5)*math.log(np.linalg.det(cov[j])) - (0.5)*np.dot( testing_data[i]-mu[j], np.dot( np.linalg.inv(cov[j]), (testing_data[i]-mu[j]).T ) )
        likelihood_list.append(likelihood)
    temp = np.array(likelihood_list)
    if np.argmax(temp) == i:
        score += 1
        print('Correct, score is now ', score)
    else:
        print('Incorrect Score for subject ', i)
print('Accuracy = ', (score*100/total_subjects))

Correct, score is now  1
Correct, score is now  2
Correct, score is now  3
Correct, score is now  4
Correct, score is now  5
Correct, score is now  6
Correct, score is now  7
Correct, score is now  8
Correct, score is now  9
Correct, score is now  10
Correct, score is now  11
Correct, score is now  12
Correct, score is now  13
Correct, score is now  14
Correct, score is now  15
Correct, score is now  16
Correct, score is now  17
Correct, score is now  18
Correct, score is now  19
Correct, score is now  20
Correct, score is now  21
Correct, score is now  22
Correct, score is now  23
Correct, score is now  24
Correct, score is now  25
Correct, score is now  26
Correct, score is now  27
Correct, score is now  28
Correct, score is now  29
Correct, score is now  30
Correct, score is now  31
Correct, score is now  32
Correct, score is now  33
Correct, score is now  34
Correct, score is now  35
Correct, score is now  36
Correct, score is now  37
Correct, score is now  38
Correct, score is now

NameError: name 'total_subjects' is not defined

In [160]:
total_subjects = 200

In [161]:
print('Accuracy = ', (score*100/total_subjects))

Accuracy =  98.5
